In [1]:
import os
from scipy.interpolate import interp1d
import pandas as pd
import plotly.express as px
import numpy as np
import geopy

import airbnb_dataset
import config_data

COVID_DATA_DIR = './COVID-19Data/Data/'

In [2]:
ctry_cities = list(config_data.country_to_city.values())
state_cities = []
states = []
for k,v in config_data.US_state_to_city.items():
    state_cities.extend(v)
    for t in range(len(v)):
        states.append(k)
print(list(zip(state_cities,states)))

[('Asheville', 'North Carolina'), ('Austin', 'Texas'), ('Boston', 'Massachusetts'), ('Cambridge', 'Massachusetts'), ('Broward County', 'Florida'), ('Chicago', 'Illinois'), ('Clark County', 'Nevada'), ('Columbus', 'Ohio'), ('Denver', 'Colorado'), ('Hawaii', 'Hawaii'), ('Jersey City', 'New Jersey'), ('Los Angeles', 'California'), ('Oakland', 'California'), ('Pacific Grove', 'California'), ('San Diego', 'California'), ('San Francisco', 'California'), ('San Mateo County', 'California'), ('Santa Clara County', 'California'), ('Santa Cruz County', 'California'), ('Nashville', 'Tennessee'), ('New Orleans', 'Louisiana'), ('New York City', 'New York'), ('Portland', 'Oregon'), ('Salem', 'Oregon'), ('Rhode Island', 'Rhode Island'), ('Seattle', 'Washington'), ('Twin Cities MSA', 'Minnesota'), ('Washington D.C.', 'District of Columbia')]


In [3]:
#weekly, 10 months
alldata = []
cols = ["city"]

min_n_months = 1000
for i,c in enumerate(state_cities):
    filepath = os.path.join(c,'covid_representative.csv')
    df = pd.read_csv(os.path.join(COVID_DATA_DIR,filepath))
    cases = df.new_cases.values
    rel_diff = [c]
    rel_diff.extend(list(np.log1p(cases)))
    alldata.append(tuple(rel_diff))
for i in range(1,44):
    cols.append("week"+str(i))
weeklycoviddf = pd.DataFrame(alldata,columns = cols)
weeklycoviddf.to_csv(os.path.join(COVID_DATA_DIR,'US/weekly_covid.csv'))
weeklycoviddf.head()

,city,week1,week2,week3,week4,week5,week6,week7,week8,week9,...,week34,week35,week36,week37,week38,week39,week40,week41,week42,week43
0,Asheville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.226411,9.307013,9.284984,8.991687,9.064852,9.532424,9.221972,9.545097,9.585896,9.551942
1,Austin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.644377,10.398732,10.238423,9.958212,10.278528,10.749334,10.334068,10.188102,10.351437,10.560930
2,Boston,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.678326,7.962067,7.645398,7.725330,7.906547,8.040125,8.367765,8.283241,8.467583,8.840146
3,Cambridge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.678326,7.962067,7.645398,7.725330,7.906547,8.040125,8.367765,8.283241,8.467583,8.840146
4,Broward County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.200365,9.926569,10.102912,9.759617,9.863030,9.713779,9.649756,9.779341,9.907380,10.050268


In [22]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

In [33]:
weeklyreviewdf = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'GLOBAL/weekly_reviews.csv'))
weeklyreviewdf.set_index(['city'],inplace=True)
weeklyreviewdf.drop(["Unnamed: 0"], axis=1, inplace =True)
weeklycoviddf = pd.read_csv(os.path.join(COVID_DATA_DIR,'GLOBAL/weekly_covid.csv'))
weeklycoviddf.set_index(['city'], inplace=True)
weeklycoviddf.drop(["Unnamed: 0"], axis=1, inplace=True)

for city in ctry_cities:
    covid_data = weeklycoviddf.loc[city].values
    review_data = weeklyreviewdf.loc[city].values[:-2]
#     if city == "Shanghai":
#         covid_data = weeklycoviddf.loc[city].values + weeklycoviddf.loc["Tokyo"].values + weeklycoviddf.loc["Singapore"].values
    x = np.vstack([covid_data,review_data])
    if city == "Shanghai":
        print(city, crosscorr(weeklycoviddf.loc[city]+weeklycoviddf.loc["Tokyo"]+weeklycoviddf.loc["Singapore"],weeklyreviewdf.loc[city]))
    print(city, crosscorr(weeklycoviddf.loc[city],weeklyreviewdf.loc[city]))
#     print(city,np.corrcoef(x)[0,1])

Munich 0.7412562753858121
Madrid 0.8672914640855702
Athens 0.6933358236658648
Stockholm 0.9030266241560423
Edinburgh 0.8442520079919029
Shanghai 0.3635837622290792
Shanghai 0.45741278065023694
Singapore 0.7415102081043254
Tokyo 0.8423193667584866
Cape Town 0.8722523497141919
Sydney 0.6753545092281105
Ottawa 0.9013288051374753
Washington D.C. 0.9139612382118948
Mexico City 0.8709683003429621
Rio de Janeiro 0.7119786655386079
Buenos Aires 0.9187710434724461


In [30]:
weeklyreviewdf = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/weekly_reviews.csv'))
weeklyreviewdf.set_index(['city'],inplace=True)
weeklyreviewdf.drop(["Unnamed: 0"], axis=1, inplace =True)
weeklycoviddf = pd.read_csv(os.path.join(COVID_DATA_DIR,'US/weekly_covid.csv'))
weeklycoviddf.set_index(['city'], inplace=True)
weeklycoviddf.drop(["Unnamed: 0"], axis=1, inplace=True)

for city in state_cities:
    covid_data = weeklycoviddf.loc[city].values
    review_data = weeklyreviewdf.loc[city].values
    x = np.vstack([covid_data,review_data])
    print(city, crosscorr(weeklycoviddf.loc[city],weeklyreviewdf.loc[city]))
#     print(city,np.corrcoef(x)[0,1])

Asheville 0.47905012884020004
Austin 0.6068620159771662
Boston 0.9380639048263754
Cambridge 0.9621774689688555
Broward County 0.4919480013083252
Chicago 0.9186601331243809
Clark County 0.4122577897216307
Columbus 0.7416943623591934
Denver 0.7667874714842732
Hawaii 0.7721629628561067
Jersey City 0.9563692565070769
Los Angeles 0.8206333865215744
Oakland 0.8309049787264813
Pacific Grove 0.5585568072724484
San Diego 0.47335309272462595
San Francisco 0.8614138544023984
San Mateo County 0.7624358993974576
Santa Clara County 0.7860958476899851
Santa Cruz County 0.5576139709699109
Nashville 0.7554839712587689
New Orleans 0.8291245343535937
New York City 0.9610925969829992
Portland 0.7641864874382989
Salem 0.6833419424645899
Rhode Island 0.7911747409695975
Seattle 0.8569312822395527
Twin Cities MSA 0.799006837635166
Washington D.C. 0.9151272444432295
